In [1]:
from datasets import load_dataset
import pandas as pd

train_dataset = load_dataset("swan07/authorship-verification", split="train[:80000]")
test_dataset = load_dataset("swan07/authorship-verification", split="test[:16000]")

from specter import SpecterVectorizer

spectorVectorizer = SpecterVectorizer()
spectorVectorizer.load()

train_spector = spectorVectorizer.embed(train_dataset)
test_spector = spectorVectorizer.embed(test_dataset)

from bert import BertVectorizer

bertVectorizer = BertVectorizer()
bertVectorizer.load()

train_bert = bertVectorizer.embed(train_dataset)
test_bert = bertVectorizer.embed(test_dataset)



c:\Users\mamy0\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 16000/16000 [02:06<00:00, 126.00it/s]


In [2]:
from pathlib import Path
import pickle

Path("./embeddings").mkdir(exist_ok=True)

# 저장
print("저장 중...")
with open('./embeddings/train_specter.pkl', 'wb') as f:
    pickle.dump(train_spector, f)
print("✓ train_specter.pkl")

with open('./embeddings/test_specter.pkl', 'wb') as f:
    pickle.dump(test_spector, f)
print("✓ test_specter.pkl")

with open('./embeddings/train_bert.pkl', 'wb') as f:
    pickle.dump(train_bert, f)
print("✓ train_bert.pkl")

with open('./embeddings/test_bert.pkl', 'wb') as f:
    pickle.dump(test_bert, f)
print("✓ test_bert.pkl")


저장 중...
✓ train_specter.pkl
✓ test_specter.pkl
✓ train_bert.pkl
✓ test_bert.pkl


In [2]:
from pathlib import Path
import pickle

with open('./embeddings/train_bert.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('./embeddings/test_bert.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [5]:
from classifier import train_authorship_classifiers
models, results = train_authorship_classifiers(train_data, test_data, True)

Using Device: cuda
Preparing features on GPU…


KeyboardInterrupt: 

In [ ]:
results

In [ ]:
from classifier import MajorityVotingEnsemble
import pandas as pd

majority_voting = MajorityVotingEnsemble(models.keys())
df_pred = pd.DataFrame({
    "true_label": test_data["same"],  # 정답 레이블
    "pred_lightgbm": results.iloc[0]["pred_test"],
    "pred_xgboost": results.iloc[1]["pred_test"],
    "pred_catboost": results.iloc[2]["pred_test"],
    "pred_simple_nn": results.iloc[3]["pred_test"],
})
majority_voting.evaluate(df_pred)